In [1]:
! pip install pandasql

import pandas as pd
from pandasql import sqldf
from random import choice, random, sample



In [2]:
major_choices = ['English', 'Science', 'Mathematics', 'Physics', 'Computer Science', 'History', 'Chemistry', 'Biology', 'Art', 'Sociology']
year_choice = [1, 2, 3, 4, 4]

major_gpa_avg = {major : random() + 2.5 for major in major_choices}

n = 100000

names = [''.join([ choice('abcdefghijklmnopqrstuvwxyz') for _ in range(7) ]) for _ in range(n)]
majors = [choice(major_choices) for _ in range(n)]
gpas = [min(major_gpa_avg[m] + random()/4.0, 4.0) for m in majors]
years = [choice(year_choice) for _ in range(n)]

student_data = pd.DataFrame( data = zip(names, gpas, majors, years), columns=['Name', 'GPA', 'Major', 'Year'] )

student_data['GPA'] = student_data.GPA.round(2)


job_choices = ['Dining','Library','AV Setup','Research Assistant','Tech Support']
wage_dic = {field:12+2*random() for field in job_choices}


n = 5000

names = sample(names,n)
job_area = [choice(job_choices) for _ in range(n)]
wage = [wage_dic[j]+random() for j in job_area]

job_data = pd.DataFrame(data = zip(names, job_area, wage), columns=['Name','Job','Wage'])

In [3]:
statement = "SELECT * FROM job_data LIMIT 20;"
selected_data = sqldf(statement, globals())
print(selected_data)

       Name                 Job       Wage
0   mqdunnq            AV Setup  14.599566
1   idkjtku              Dining  12.751683
2   adqlfaq              Dining  13.109625
3   kertldy             Library  13.225989
4   pglmruc  Research Assistant  14.076964
5   dhawhti            AV Setup  14.084629
6   aqvdsri  Research Assistant  14.261009
7   eawtcrz             Library  13.669537
8   ccxxtgm  Research Assistant  14.176123
9   tishhry              Dining  12.966573
10  lrpzeww             Library  13.374574
11  uhqqrqo            AV Setup  14.130457
12  xutefkc  Research Assistant  13.859788
13  bapxuxf              Dining  12.656123
14  ajpjwlt             Library  13.342923
15  kqglvcz            AV Setup  14.268937
16  uqkctux            AV Setup  14.676703
17  ilbaugb             Library  13.275947
18  jmkfnbz            AV Setup  13.824317
19  xyghurm              Dining  13.477196


In [4]:
#How does average GPA compare across Jobs

In [5]:
statement = " \
    SELECT S.Name, Job, GPA \
    FROM student_data S \
    INNER JOIN job_data J ON S.Name =J.Name;"
selected_data = sqldf(statement, globals())
print(selected_data)

         Name                 Job   GPA
0     ubwmyfy  Research Assistant  3.28
1     jngzxag             Library  3.04
2     dspozch              Dining  3.26
3     qhbzaly              Dining  3.50
4     zgtfzgw             Library  2.86
...       ...                 ...   ...
4995  jpgydkj        Tech Support  2.96
4996  euakdww  Research Assistant  2.86
4997  tzlsvjt  Research Assistant  2.88
4998  tyzufhk        Tech Support  2.97
4999  vdjvrkg              Dining  3.22

[5000 rows x 3 columns]


In [6]:
statement = " \
    SELECT Job, AVG(GPA) as AvgGPA \
    FROM student_data S \
    INNER JOIN job_data J ON S.Name =J.Name \
    GROUP BY Job \
    ORDER BY AvgGPA DESC;"
selected_data = sqldf(statement, globals())
print(selected_data)

                  Job    AvgGPA
0             Library  3.104801
1            AV Setup  3.104344
2        Tech Support  3.098150
3  Research Assistant  3.096170
4              Dining  3.094694


In [7]:
#What fraction of each major have jobs

#Step 1 Left Join the two tables

statement = " \
    SELECT Major, Job \
    FROM student_data S \
    LEFT JOIN job_data J ON S.Name =J.Name;"
selected_data = sqldf(statement, globals())
print(selected_data)

                  Major                 Job
0      Computer Science                None
1           Mathematics                None
2               Science                None
3             Chemistry                None
4               Science  Research Assistant
...                 ...                 ...
99995           History        Tech Support
99996           Physics                None
99997           Science              Dining
99998           Science                None
99999       Mathematics                None

[100000 rows x 2 columns]


In [8]:
#Group the results above my major

statement = " \
    SELECT Major, SUM(CASE WHEN Job IS NOT NULL THEN 1.0 ELSE 0.0 END)/COUNT(*) AS fracwork\
    FROM student_data S \
    LEFT JOIN job_data J ON S.Name =J.Name \
    GROUP BY Major;"
selected_data = sqldf(statement, globals())
print(selected_data)

              Major  fracwork
0               Art  0.052442
1           Biology  0.047305
2         Chemistry  0.052351
3  Computer Science  0.055132
4           English  0.050438
5           History  0.048429
6       Mathematics  0.048004
7           Physics  0.046224
8           Science  0.051317
9         Sociology  0.048414


In [9]:
#STEP 3 

statement = " \
    SELECT Major, SUM(CASE WHEN Job IS NOT NULL THEN 1.0 ELSE 0.0 END)/COUNT(*) AS fracwork\
    FROM student_data S \
    LEFT JOIN job_data J ON S.Name =J.Name \
    GROUP BY Major \
    HAVING fracwork >= 0.05 \
    ORDER BY fracwork DESC;"
selected_data = sqldf(statement, globals())
print(selected_data)

              Major  fracwork
0  Computer Science  0.055132
1               Art  0.052442
2         Chemistry  0.052351
3           Science  0.051317
4           English  0.050438


SELF JOINS

In [10]:
#STEP 1: Perform a SELF JOIN of job table on itself

statement = " \
    SELECT J1.Name AS Name_1, J2.Name AS Name_2, \
    J1.Job AS Job_1, J2.Job AS Job_2, ABS(J1.Wage - J2.Wage) AS AbsWageDiff \
    FROM job_data J1, job_data J2 \
    WHERE J1.Name != J2.Name \
    LIMIT 20;"

selected_data = sqldf(statement, globals())
print(selected_data)

     Name_1   Name_2     Job_1               Job_2  AbsWageDiff
0   mqdunnq  idkjtku  AV Setup              Dining     1.847883
1   mqdunnq  adqlfaq  AV Setup              Dining     1.489941
2   mqdunnq  kertldy  AV Setup             Library     1.373577
3   mqdunnq  pglmruc  AV Setup  Research Assistant     0.522602
4   mqdunnq  dhawhti  AV Setup            AV Setup     0.514937
5   mqdunnq  aqvdsri  AV Setup  Research Assistant     0.338557
6   mqdunnq  eawtcrz  AV Setup             Library     0.930029
7   mqdunnq  ccxxtgm  AV Setup  Research Assistant     0.423442
8   mqdunnq  tishhry  AV Setup              Dining     1.632992
9   mqdunnq  lrpzeww  AV Setup             Library     1.224992
10  mqdunnq  uhqqrqo  AV Setup            AV Setup     0.469109
11  mqdunnq  xutefkc  AV Setup  Research Assistant     0.739778
12  mqdunnq  bapxuxf  AV Setup              Dining     1.943443
13  mqdunnq  ajpjwlt  AV Setup             Library     1.256643
14  mqdunnq  kqglvcz  AV Setup          

In [11]:
statement = ' \
    SELECT AVG(ABS(J1.Wage - J2.Wage)) As AvgAbsWageDiff \
    FROM job_data J1, job_data J2 \
    WHERE J1.Name != J2.Name'

selected_data = sqldf(statement, globals())
print(selected_data)

   AvgAbsWageDiff
0        0.743811


In [12]:
#How does this Wage gap compare to wage gaps within each job?


In [13]:
statement = ' \
    SELECT J1.Name AS Name_1, J2.Name AS Name_2, J1.Job AS Job, \
        ABS(J1.Wage - J2.Wage) AS AbsWageDiff \
    FROM job_data J1, job_data J2 \
    WHERE J1.Name != J2.Name AND J1.Job = J2.Job \
    LIMIT 20;'

selected_data = sqldf(statement, globals())
print(selected_data)

     Name_1   Name_2       Job  AbsWageDiff
0   mqdunnq  aaeyrkt  AV Setup     0.133834
1   mqdunnq  aanicag  AV Setup     0.058705
2   mqdunnq  aauvbjp  AV Setup     0.469697
3   mqdunnq  aaxtays  AV Setup     0.249647
4   mqdunnq  abifkbg  AV Setup     0.182585
5   mqdunnq  abnlbzh  AV Setup     0.740760
6   mqdunnq  acgggms  AV Setup     0.092887
7   mqdunnq  adkewai  AV Setup     0.661531
8   mqdunnq  adnksoe  AV Setup     0.086576
9   mqdunnq  afbyqdy  AV Setup     0.027493
10  mqdunnq  afcaony  AV Setup     0.655134
11  mqdunnq  afivuvg  AV Setup     0.605403
12  mqdunnq  aggwouw  AV Setup     0.698872
13  mqdunnq  agipdau  AV Setup     0.094091
14  mqdunnq  agvxgtv  AV Setup     0.373764
15  mqdunnq  agwxbrz  AV Setup     0.579456
16  mqdunnq  ahczjom  AV Setup     0.219237
17  mqdunnq  ahwubcs  AV Setup     0.217475
18  mqdunnq  ajawltk  AV Setup     0.148661
19  mqdunnq  ajfrjyu  AV Setup     0.174177


In [14]:
statement = ' \
    SELECT J1.Job AS Job, \
        ABS(J1.Wage - J2.Wage) AS AbsWageDiff \
    FROM job_data J1, job_data J2 \
    WHERE J1.Name != J2.Name AND J1.Job = J2.Job \
    GROUP BY J1.Job;'

selected_data = sqldf(statement, globals())
print(selected_data)

                  Job  AbsWageDiff
0            AV Setup     0.133834
1              Dining     0.192404
2             Library     0.211843
3  Research Assistant     0.055638
4        Tech Support     0.428913
